# Tratamento de dados

In [9]:
# Importação de bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd

Utilizaremos os dados dos sites 
- [x] [CBF - Série A](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a)
- [x] [CBF - Série B](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b)
- [x] [CBF - Série C](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-c)

In [10]:
# URLs para web scraping:
url_serie_a = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a'
url_serie_b = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b'
url_serie_c = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-c'

# Fazendo o request para o site:
request_brasileirao_a = requests.get(url_serie_a)
request_brasileirao_b = requests.get(url_serie_b)
request_brasileirao_c = requests.get(url_serie_c)

# Utilizando BeautifulSoup para tratamento HTML:
soup_a = BeautifulSoup(request_brasileirao_a.text, 'html.parser')
soup_b = BeautifulSoup(request_brasileirao_b.text, 'html.parser')
soup_c = BeautifulSoup(request_brasileirao_c.text, 'html.parser')

# Achando as tabelas com as informações:
tabela_a = soup_a.find(class_='table m-b-20 tabela-expandir')
tabela_b = soup_b.find(class_='table m-b-20 tabela-expandir')
tabela_c = soup_c.find(class_='table m-b-20 tabela-expandir')

## Realizando a coleta dos dados:

### Tabela Série A

In [49]:
# ---------------- TABELA SÉRIE A ----------------
# Obtendo o título das colunas e a pontuação com <th>
classes_e_pontuacao_a = []
for i in tabela_a.find_all('th'):
    title = i.text
    classes_e_pontuacao_a.append(title)

categorias_a = classes_e_pontuacao_a[:-20]
pontos_a = classes_e_pontuacao_a[-20:]

dados_tabela_a = []
# Criando um loop for para preencher os meus dados
for j in tabela_a.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    dados_tabela_a.append(row)

tabela_principal = dados_tabela_a[::2]
tabela_info = dados_tabela_a[1::2]

# Tratamento da tabela principal
categorias_a.insert(1, 'Time')
for dado in tabela_principal:
    # Colocando Posição e Time
    dado[0] = dado[0].replace('\n', '').replace('\xa00', '').replace('+1', '').replace('-1', '')
    dado[0] = dado[0].replace('+2', '').replace('-2', '').replace('+3', '').replace('-3', '')
    dado.insert(0, dado[0].split('º')[0])
    dado.insert(1, dado[1].split('º')[1])
    dado.pop(2)
    # Jogos Recentes
    dado[12] = dado[12][1:-1].replace('\n', ',')

# Colocando os Pontos nos dados:
for indice, pontuacao in enumerate(pontos_a):
    tabela_principal[indice].insert(2, pontuacao)

DadosBrasileirao_a = pd.DataFrame(columns=categorias_a, data=tabela_principal)
print(DadosBrasileirao_a)


   Posição                       Time PTS   J   V   E   D  GP  GC   SG   CA  \
0        1      Atlético Mineiro - MG  84  38  26   6   6  67  34   33   82   
1        2              Flamengo - RJ  71  38  21   8   9  69  36   33   82   
2        3             Palmeiras - SP  66  38  20   6  12  58  43   15   87   
3        4             Fortaleza - CE  58  38  17   7  14  44  45   -1   95   
4        5           Corinthians - SP  57  38  15  12  11  40  36    4   61   
5        6   Red Bull Bragantino - SP  56  38  14  14  10  55  46    9   95   
6        7            Fluminense - RJ  54  38  15   9  14  38  38    0   91   
7        8               America - MG  53  38  13  14  11  41  37    4   66   
8        9              Atlético - GO  53  38  13  14  11  33  36   -3   98   
9       10                Santos - SP  50  38  12  14  12  35  40   -5  103   
10      11                 Ceará - CE  50  38  11  17  10  39  38    1   90   
11      12         Internacional - RS  48  38  12  1

### Tabela Série B

In [50]:
# ---------------- TABELA SÉRIE B ----------------
# Obtendo o título das colunas e a pontuação com <th>
classes_e_pontuacao_b = []
for i in tabela_b.find_all('th'):
    title = i.text
    classes_e_pontuacao_b.append(title)

categorias_b = classes_e_pontuacao_b[:-20]
pontos_b = classes_e_pontuacao_b[-20:]

dados_tabela_b = []
# Criando um loop for para preencher os meus dados
for j in tabela_b.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    dados_tabela_b.append(row)

tabela_principal = dados_tabela_b[::2]
tabela_info = dados_tabela_b[1::2]

# Tratamentoda Posição e Time
categorias_b.insert(1, 'Time')
for dado in tabela_principal:
    # Colocando Posição e Time
    dado[0] = dado[0].replace('\n', '').replace('\xa00', '').replace('+1', '').replace('-1', '')
    dado[0] = dado[0].replace('+2', '').replace('-2', '').replace('+3', '').replace('-3', '')
    dado.insert(0, dado[0].split('º')[0])
    dado.insert(1, dado[1].split('º')[1])
    dado.pop(2)
    # Jogos Recentes
    #dado[12] = dado[12][1:-1].replace('\n', ',')

for indice, pontuacao in enumerate(pontos_b):
    tabela_principal[indice].insert(2, pontuacao)

DadosBrasileirao_b = pd.DataFrame(columns=categorias_b, data=tabela_principal)
print(DadosBrasileirao_b)

                                              Posição  \
0                                                   1   
1                                                   2   
2                                                   3   
3                                                   4   
4                                                   5   
5                                                   6   
6                                                   7   
7                                                   8   
8                                                   9   
9                                                  10   
10                                                 11   
11                                                 12   
12                                                 13   
13                                                 14   
14                                                 15   
15                                                 16   
16                             

### Tabela Série C

In [33]:
# ---------------- TABELA SÉRIE C ----------------
# Obtendo o título das colunas e a pontuação com <th>
classes_e_pontuacao_c = []
for i in tabela_c.find_all('th'):
    title = i.text
    classes_e_pontuacao_c.append(title)

categorias_c = classes_e_pontuacao_c[:-20]
pontos_c = classes_e_pontuacao_c[20:]

dados_tabela_c = []
# Criando um loop for para preencher os meus dados
for j in tabela_c.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    dados_tabela_c.append(row)

tabela_principal = dados_tabela_c[::2]
tabela_info = dados_tabela_c[1::2]

# Tratamentoda Posição e Time
categorias_a.insert(1, 'Time')
for dado in tabela_principal:
    dado[0] = dado[0].replace('\n', '').replace('\xa00', '').replace('+1', '').replace('-1', '')
    dado[0] = dado[0].replace('+2', '').replace('-2', '').replace('+3', '').replace('-3', '')
    dado.insert(0, dado[0].split('º')[0])
    dado.insert(1, dado[1].split('º')[1])
    dado.pop(2)

print(categorias_c)
print(tabela_principal)

[]
[['1', 'Ituano - SP', '2', '1', '1', '0', '4', '1', '3', '6', '0', '66', '\nV\nE\nV\n', '\n'], ['2', 'Tombense - MG', '2', '0', '1', '1', '1', '4', '-3', '4', '1', '16', '\nE\nE\nD\n', '\n']]
